# Algorithms and mathematics of machine learning
## Laboratory 11 - inverse matrix and matrix determinant

### Task 1 - Hill's cipher

Implement two functions to encode and decode messages using the [Hill](https://en.wikipedia.org/wiki/Hill_cipher) cipher:
* `encrypt(t, K)` - encrypts the text $t$ using the key (matrix) $K$.
* `decrypt(s, K)` - decrypts the $s$ code using the key (matrix) $K$.

#### Coding

In the ASCII coding system, the letters A-Z are written as numbers in the range 65-90. To encrypt the text using the key $K$ (matrix with dimensions $m$ x $m$), you must save the text characters in the form of a matrix with dimensions $m$ x $n$ and then perform the following operations:
1. Create an encryption matrix $K$ whose determinant is $det(K) = 1$.
   > Note: This is a departure from the original algorithm to simplify the example.
   > Such a matrix can be created from an identity matrix (`np.identity`), using elementary operations, e.g. adding to one row of the matrix another row multiplied by a scalar.
2. Convert the text letters $t$ of length $h$ into a vector of numbers.
3. Pad (_padding_) with zeros so that the next step can be performed.
4. Transform into a $X$ matrix of dimensions ($m$ x $n$), where $n = \lceil \frac{h}{m} \rceil$ (you can use the `reshape` function).
5. Perform the operation $S = (KX)$.
6. Convert the matrix $S$ to a vector (you can use the `flatten` function) and return the cryptogram $s$ (encrypted text).

> Note 1: The presented algorithm is a simplified algorithm with the limit $det(K)=1$, which can be omitted, but then the inverse modulo 26 matrix must be determined for the coding matrix (the number of characters A-Z, but it can be any other). Similarly, the matrix $S$ should be changed to modulo 26.<br>**Important**: In this case, remember that the determinant of the encryption matrix $det(K)$ cannot have a common divisor with the number 26 (i.e. both numbers must be relatively prime). Why? Because otherwise there is no inverse number to $det(K) \textit{ mod } 26$.

> Note 2: the string $s$ may contain unprintable characters. If you want to avoid this, you can, for example, map the characters (65-90) to the range 0-25. Then perform the operation in the opposite direction while displaying.

#### Decoding

To decrypt an encrypted message $s$:
1. Convert the string $s$ into a matrix $S$.
2. Calculate the inverse matrix $K^{-1}$ (function `np.linalg.inv`).
3. Decrypt the message by performing the operation $W = K^{-1} S$.
4. Convert the message $W$ to the text string $w$.

> Note: This is only a laboratory example of the use of matrix operations. The presented solution is not safe.

In [1]:
from numpy import ndarray, array, identity, append
from numpy.linalg import inv, det


def encrypt(t: str, k: int) -> tuple:
    code = identity(k) * 10
    text = array([ord(letter) for letter in list(t)])

    if text.shape[0] % k > 0:
        for _ in range(k - text.shape[0] % k):
            text = append(text, array([0]))

    text = text.reshape((k, int(text.shape[0] / k)))
    s = code @ text

    return ''.join([chr(txt) for txt in s.flatten()[:len(t)].astype(int)]), code


def decrypt(s: str, k: ndarray) -> str:
    text = array([ord(ss) for ss in list(s)])

    if text.shape[0] % k.shape[0] > 0:
        for _ in range(k.shape[0] - text.shape[0] % k.shape[0]):
            text = append(text, array([0]))

    text = text.reshape((k.shape[0], int(text.shape[0] / k.shape[0])))
    decoded = inv(k) @ text

    return ''.join([chr(txt) for txt in decoded.flatten()[:len(s)].astype(int)])

In [2]:
text = 'example text'

In [3]:
t_code, code = encrypt(text, 3)
t_code

'ϲҰϊтѠиϲŀ҈ϲҰ҈'

In [4]:
decrypt(t_code, code)

'example text'

### Task 2

Implement a function that takes a square matrix as an argument and returns its determinant calculated according to [Leibniz](https://en.wikipedia.org/wiki/Leibniz_formula_for_determinants) (_permutation definition_) and compare with the result of the ready function from the numpy library `np.linalg.det`.

$$
\text{det}(A) = \sum_{\sigma \in S_n}\left(\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}\right)
$$

where:

* $S_n$ - [symmetric group](https://en.wikipedia.org/wiki/Symmetric_group) (for a 3x3 matrix these will be permutations from the set {0, 1, 2})
* $\text{sgn}$ - this is the symbol "+", "-" depending on the [parity of the permutation](https://en.wikipedia.org/wiki/Parity_of_a_permutation). E.g. for the permutation `[1, 2, 0]` it will be "+" (you need to perform two operations - replace `0` with `1` and then `0` with `2`, and for the permutation `[0, 2 , 1]` will be "-" because one operation is enough (replacing `1` with `2`).
* $\sigma$ - permutation (element from the permutation group $S_n$)

##### Example for a 3x3 matrix

| $\sigma$  | $\text{sgn}$  | $\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}$ |
|:----------|:--------------|------------------------------------------------------:|
| 1, 2, 3   | +             |                              $+a_{1,1}a_{2,2}a_{3,3}$ |
| 1, 3, 2   | -             |                              $-a_{1,1}a_{2,3}a_{3,2}$ |
| 3, 1, 2   | +             |                              $+a_{1,3}a_{2,1}a_{3,2}$ |
| 3, 2, 1   | -             |                              $-a_{1,3}a_{2,2}a_{3,1}$ |
| 2, 3, 1   | +             |                              $+a_{1,2}a_{2,3}a_{3,1}$ |
| 2, 1, 3   | -             |                              $-a_{1,2}a_{2,1}a_{3,3}$ |

$\text{det}(A) = a_{1,1}a_{2,2}a_{3,3} - a_{1,1}a_{2,3}a_{3,2} + a_{1,3}a_{2,1}a_{3,2} - a_{1,3}a_{2,2}a_{3,1} + a_{1,2}a_{2,3}a_{3,1} - a_{1,2}a_{2,1}a_{3,3}$

> Note 1: To check the parity of permutations you can use the `parity` function from the [`sympy`](https://docs.sympy.org/latest/modules/combinatorics/permutations.html#sympy.combinatorics.permutations.Permutation.parity). Example: 
> ```python
> from sympy.combinatorics import Permutation
> Permutation([0, 2, 1]).parity()
> ```

> Note 2: To generate permutations you can use the `permutations` function from the [`itertools`](https://docs.python.org/3/library/itertools.html#itertools.permutations) module

> Note 3: Remember that in numpy, comparing floating point numbers is done using the function [`allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html)

In [5]:
from numpy.random import randint
from numpy import prod, allclose
from itertools import permutations
from sympy.combinatorics import Permutation


def determinant(matrix: ndarray) -> float:
    idx = list(range(matrix.shape[0]))
    result = 0.0

    for perm in permutations(idx):
        val = -1 if Permutation(perm).parity() == 1 else 1
        result += val * prod([matrix[i, perm[i]] for i in idx])

    return result


mat = randint(low=0, high=10, size=(4, 4))
print(allclose(determinant(mat), det(mat)))

True
